<a href="https://colab.research.google.com/github/Aigiun/public/blob/main/MetaZee_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-west-1 region. You will use the 632365934929.dkr.ecr.us-west-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [ ]:
bucket_name = 'predictionenglandleague123' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [ ]:
try:
  urllib.request.urlretrieve ("https://drive.google.com/uc?export=download&id=18hvXO38OXjjPF6Sqm2SM1-C3r54qppxD", "example.csv")
  print('Success: downloaded example.csv.')
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./example.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded example.csv.
Success: Data loaded into dataframe.


In [ ]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(3154, 28) (1352, 28)


In [ ]:
pd.concat([train_data['FTR_H'], train_data.drop(['FTR_D', 'FTR_H'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [ ]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1, instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [ ]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2023-05-01-06-34-29-749


2023-05-01 06:34:29 Starting - Starting the training job...
2023-05-01 06:34:55 Starting - Preparing the instances for training......
2023-05-01 06:35:49 Downloading - Downloading input data...
2023-05-01 06:36:19 Training - Downloading the training image...
2023-05-01 06:36:54 Training - Training image download completed. Training in progress...Arguments: train
[2023-05-01:06:37:05:INFO] Running standalone xgboost training.
[2023-05-01:06:37:05:INFO] Path /opt/ml/input/data/validation does not exist!
[2023-05-01:06:37:05:INFO] File size need to be processed in the node: 0.24mb. Available memory size in the node: 8587.41mb
[2023-05-01:06:37:05:INFO] Determined delimiter of CSV input is ','
[06:37:05] S3DistributionType set as FullyReplicated
[06:37:05] 3154x26 matrix with 82004 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[06:37:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 8 pruned nodes, max_depth=5
[0]#011train-err

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2023-05-01-06-40-24-482
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-05-01-06-40-24-482
INFO:sagemaker:Creating endpoint with name xgboost-2023-05-01-06-40-24-482


------!

In [ ]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(['FTR_D', 'FTR_H'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(1352,)


In [ ]:
cm = pd.crosstab(index=test_data['FTR_H'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "win", "not win"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("win", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("not win", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 84.1%

Predicted      win             not win
Observed
win            85% (637)    17% (100)
not win         15% (115)     83% (500) 



In [ ]:
test_data['predictions'] = predictions

In [ ]:
test_df['prediction'] = result_df.values

In [ ]:
test_data.to_excel('test_data.xlsx', index=False)